In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg

StatementMeta(, 9c476edc-390b-415e-9c25-a65ed36a7338, 10, Finished, Available, Finished)

In [7]:
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)

StatementMeta(, 9c476edc-390b-415e-9c25-a65ed36a7338, 11, Finished, Available, Finished)

In [8]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 9c476edc-390b-415e-9c25-a65ed36a7338, 12, Finished, Available, Finished)

In [9]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 9c476edc-390b-415e-9c25-a65ed36a7338, 13, Finished, Available, Finished)

In [10]:
# adding country_code and city attributes
df_with_location = (
df
.withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))
)

StatementMeta(, 9c476edc-390b-415e-9c25-a65ed36a7338, 14, Finished, Available, Finished)

In [11]:
# adding significance classification
df_with_location_sig_class =(
df_with_location
.withColumn('sig_class', when(col("sig") < 100, "Low")
                        .when((col("sig") >= 100) & (col("sig") < 500), "Moderate")
                         .otherwise("High")
            )
)

StatementMeta(, 9c476edc-390b-415e-9c25-a65ed36a7338, 15, Finished, Available, Finished)

In [12]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, 9c476edc-390b-415e-9c25-a65ed36a7338, 16, Finished, Available, Finished)